In [1]:
import numpy as np
import pandas as pd
import scanpy.api as sc
import anndata



import os
import scipy
from scipy import sparse
import sys

%matplotlib inline 
import matplotlib.pyplot as plt
import datetime as datetime

import tensorflow as tf
import keras
import tensorboard
from keras.layers import Dense, Flatten, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP
import seaborn as sns 

sc.settings.verbosity = 3 
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures
sc.logging.print_versions()


learning_rate = 0.005
learning_rate_g = 0.001



/home/emcandrew/anaconda3/envs/tf_latest/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,
Using TensorFlow backend.


scanpy==1.5.1 anndata==0.7.3 umap==0.4.4 numpy==1.18.1 scipy==1.4.1 pandas==1.0.3 scikit-learn==0.23.1 statsmodels==0.11.1 python-igraph==0.8.2 louvain==0.7.0


# open tensorboard and set log dir 

In [2]:
log_dir = "./tmp/logs/"
summary_writer = tf.summary.create_file_writer(logdir=log_dir)
%load_ext tensorboard
%tensorboard --logdir ./tmp/logs/

Reusing TensorBoard on port 6006 (pid 3368), started 4 days, 0:05:44 ago. (Use '!kill 3368' to kill it.)

# Reading in already preprocessed input data

In [3]:
scaler = MinMaxScaler()

exp_mat = pd.read_csv("Gan_input2.csv", sep='\t', index_col = 0)
exp_mat = exp_mat.T

input_matrix = np.genfromtxt('Gan_input2.csv', skip_header=1)

input_matrix = input_matrix.T
input_matrix.shape


input_matrix = np.delete(input_matrix, (0), axis=0)

scaler.fit(input_matrix)
input_matrix = scaler.transform(input_matrix)






In [4]:
input_matrix[1].shape

(793,)

In [5]:
# Model input dimensions


cell = (793,)
z_dim = 100 # noise vector size

In [6]:
input_matrix[1]

array([0.04862233, 0.0973451 , 0.09687525, 0.07856252, 0.0147812 ,
       0.01920392, 0.81562851, 0.44044766, 0.06855113, 0.28822035,
       0.02502037, 0.43199348, 0.11830732, 0.08039244, 0.0476922 ,
       0.04408576, 0.08625219, 0.2066288 , 0.14781333, 0.27369769,
       0.07839337, 0.01028991, 0.58214049, 0.37654752, 0.02902108,
       0.01979388, 0.03205911, 0.13221802, 0.87988892, 0.06814237,
       0.10142352, 0.35575929, 0.12843781, 0.11922874, 0.05723442,
       0.03190849, 0.07722307, 0.00597578, 0.07536983, 0.0603617 ,
       0.3104535 , 0.13179429, 0.20605584, 0.04396074, 0.36566019,
       0.23292322, 0.78121021, 0.72915296, 0.76514923, 0.15905401,
       0.22161928, 0.28123768, 0.13233827, 0.06387014, 0.11978799,
       0.10004397, 0.11301476, 0.79455268, 0.11043198, 0.08118584,
       0.30215932, 0.05081357, 0.08015451, 0.34057817, 0.14005516,
       0.80641925, 0.51819116, 0.32599389, 0.09818237, 0.10035685,
       0.02478997, 0.20499261, 0.03987337, 0.03708293, 0.20437

In [7]:
# Generator as a function. Returns the generator network model.

def build_generator(cell, z_dim) :
  
  # Defines the model
  model = Sequential()

  # Adds a dense layer of 64 neurons with input_dim equal to z_dim
  model.add(Dense(450, input_dim = z_dim))

  # Apply Leaky ReLU activaion function 
  model.add(LeakyReLU(alpha=0.2))

  # Adds another fully connected layer - output layer 
  model.add(Dense(793))

  model.add(LeakyReLU(alpha=0.2))

  model.add(Reshape(cell))

  return model

In [8]:
# Defining the Discrimator network

def build_critic(cell) :

  model = Sequential()

  model.add(Dense(200, input_shape = cell))

  model.add(LeakyReLU(alpha=0.2))

  model.add(Dense(200))

  model.add(LeakyReLU(alpha=0.2))

  model.add(Dense(1))

  return model

In [9]:
# wasserstein loss function

def wasserstein( y_true, y_pred):
    wloss = -keras.backend.mean(y_true * y_pred)
    return wloss

# 
def build_gan(generator, critic) :

  model = Sequential()

  model.add(generator)
  model.add(critic)

  return model

critic = build_critic(cell)
critic.compile(loss = wasserstein,
                    optimizer = keras.optimizers.RMSprop(learning_rate = 0.00005),
                    metrics = ['accuracy'])

generator = build_generator(cell, z_dim)
critic.trainable = False

gan = build_gan(generator, critic)
gan.compile(loss= wasserstein, optimizer= keras.optimizers.RMSprop(learning_rate =  0.00005))

In [10]:
def generate_cells(number_cells) :
    
    # Draws from random normal.
    noise = np.random.normal(0, 1, (1, z_dim))
    
    # predcts using enerator network.
    gen_cell = generator.predict(noise)
    
    # Transform back from MinMax scaling between -1 and 1 (necesscary for tanh activation function) 
    gen_cell = scaler.inverse_transform(gen_cell)


   # Repeat of above procedure inside a loop. 
    for cell in range(num_cells -1 ):
    
            noise = np.random.normal(0, 1, (1, z_dim))
    
            gen_cells_tmp = generator.predict(noise)
    
            gen_cells_tmp = scaler.inverse_transform(gen_cells_tmp)
        
          #  Appends subsequent generated cells to form an array of generated cells
            gen_cell = np.append(gen_cell, gen_cells_tmp, axis=0)
    
    # Converts array to matrice object
    gen_cell = np.asmatrix(gen_cell)
    
    # converts matrice to pandas dataframe
    gen_cell = pd.DataFrame(gen_cell)
    
    # Loop to create cell names (theres deinitely a better way to do this)
    cell_names = []
    for i in range(num_cells ):
        cell_names.append("cell-{}".format(i +1))
        
        
    # Creates final dataframe with gene names and cell IDs
    gen_cell = pd.DataFrame(data=gen_cell.values, columns=exp_mat.columns, index = cell_names)
    
    return gen_cell


In [11]:
accuracies = []
iteration_checkpoints = []
Generated_cells = []



def train_critic(X_train, batch_size, clip_threshold):

    X_train =  input_matrix

    valid = np.ones((batch_size, 1))
    fake = -np.ones((batch_size, 1))
    
    # Train on real data
    idx = np.random.randint(X_train.shape[0], size = batch_size)
    real_cell = X_train[idx]
    c_loss_real = critic.train_on_batch(real_cell, valid)
    
    
    
    # Train on generated data
    noise = np.random.normal(0,1, (batch_size, z_dim))
    fake_cell = generator.predict(noise)
    c_loss_fake = critic.train_on_batch(fake_cell, fake)
    
    c_loss = 0.5 * np.add(c_loss_fake, c_loss_real)
    
      
    for l in critic.layers:
        weights = l.get_weights()
        weights = [np.clip(w, -clip_threshold, clip_threshold) for w in weights]
        l.set_weights(weights)
    
    return c_loss
    




      

In [12]:
def train_generator(batch_size) :
    valid = np.ones((batch_size, 1))
    
    noise = np.random.normal(0,1, (batch_size, z_dim))
    g_loss = gan.train_on_batch(noise, valid)
    return g_loss
    
    
    

In [13]:
# running the model

losses = []

# Setting hyper parameters
epochs = 30000
batch_size = 32
sample_interval = 1000
generate_cells_every = 3000
num_cells = 2000
X_train = input_matrix

for epoch in range(epochs):
    for _ in range(2):
        c_loss = train_critic(X_train, batch_size= batch_size, clip_threshold = 0.01)
    
    g_loss = train_generator(batch_size)

    if (epoch + 1) % sample_interval == 0: 
        iteration_checkpoints.append(epoch + 1)
        losses.append((c_loss, g_loss))
        print ("%d [D loss: %f] [G loss: %f]" % (epoch+1, 1 - sum(c_loss)/len(c_loss), 1 - g_loss))

    if (epoch +1) % generate_cells_every == 0:
        gen_cell = generate_cells(500)
        Generated_cells.append(gen_cell)

/home/emcandrew/anaconda3/envs/tf_latest/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1000 [D loss: 1.001285] [G loss: 0.974836]
2000 [D loss: 1.000520] [G loss: 0.994503]
3000 [D loss: 1.001611] [G loss: 0.983599]


/home/emcandrew/anaconda3/envs/tf_latest/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


KeyboardInterrupt: 

In [ ]:
Generated_cells[3]

In [ ]:
#output = Generated_cells[4]
#output.to_csv('gen_cells.csv', sep='\t', index=True)

In [ ]:
Generated_cells[1]

In [ ]:
for dataset in Generated_cells :
    test = dataset
    adata = sc.AnnData(test)
    adata.X
    sc.pp.neighbors(adata)
    sc.tl.umap(adata)
    sc.tl.louvain(adata, resolution=1, key_added='louvain_r1')
    sc.pl.umap(adata, color='louvain_r1')
    sc.tl.rank_genes_groups(adata, 'louvain_r1', method='logreg')
    sc.pl.rank_genes_groups(adata, n_genes=20)
    result = adata.uns['rank_genes_groups']
    groups = result['names'].dtype.names
    pd.DataFrame({group + '_' + key[:1]: result[key][group]
        for group in groups for key in ['names', 'scores']}).head(10)
        
